In [ ]:
from qiskit_aer import AerSimulator

from qlbm.components import EmptyPrimitive
from qlbm.components.spacetime import (
    PointWiseSpaceTimeInitialConditions,
    SpaceTimeGridVelocityMeasurement,
    SpaceTimeQLBM,
)
from qlbm.infra import QiskitRunner, SimulationConfig
from qlbm.lattice import SpaceTimeLattice
from qlbm.tools.utils import create_directory_and_parents

In [ ]:
# Load example with mixed boundary conditions and create output directory
lattice = SpaceTimeLattice(
    num_timesteps=1,
    lattice_data={
        "lattice": {"dim": {"x": 4, "y": 8}, "velocities": "D2Q4"},
        "geometry": [],
    },
)

output_dir = f"qlbm-output/spacetime-{lattice.logger_name()}-qiskit"
create_directory_and_parents(output_dir)

In [ ]:
cfg = SimulationConfig(
    initial_conditions=PointWiseSpaceTimeInitialConditions(
        lattice, grid_data=[((1, 5), (True, True, True, True))]
    ),
    algorithm=SpaceTimeQLBM(lattice),
    postprocessing=EmptyPrimitive(lattice),
    measurement=SpaceTimeGridVelocityMeasurement(lattice),
    target_platform="QISKIT",
    compiler_platform="QISKIT",
    optimization_level=0,
    statevector_sampling=False,
    execution_backend=AerSimulator(method="statevector"),
    sampling_backend=AerSimulator(method="statevector"),
)

In [ ]:
cfg.prepare_for_simulation()

In [ ]:
# Number of shots to simulate for each timestep when running the circuit
NUM_SHOTS = 2**10

# Number of timesteps to simulate
NUM_STEPS = 10

In [ ]:
# Create a runner object to simulate the circuit
runner = QiskitRunner(
    cfg,
    lattice,
)

In [ ]:
# Simulate the circuits using both snapshots and sampling
runner.run(
    NUM_STEPS,  # Number of time steps
    NUM_SHOTS,  # Number of shots per time step
    output_dir,
    statevector_snapshots=True,
    recompile_each_step=True,  # This is necessary due to Qiskit transpiler settings. WIP.
)